In [19]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing
import scipy.optimize

In [20]:
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2_Crack_simp import (
    GetScan,
    recursive_scan,
    Para_init,
    Run_model_wwo_dry_out ,
    Cal_new_con_Update,
    Run_Model_Base_On_Last_Solution,
    Run_Model_Base_On_Last_Solution_RPT,
    write_excel_xlsx,
    Get_initial_cap,
    Get_initial_cap2,
    handle_signal
)

In [27]:
########################  Input  ########################
# all values here must be a list, even it is a single object
Para_dict_All = {
   "Total ageing cycles":[1170,],
   "Ageing cycles between RPT":[78,],
   "Update cycles for ageing":[26,],
   "Cycles within RPT":[2,],
   "Ageing temperature":[40,],
   "RPT temperature":[25,],
   "Particle mesh points":[30,],   # Simon uses 30
   "Exponential mesh stretch":[1.0],
   "Para_Set":[ "OKane2022",],
   "sei":["OKane2022"],
   #"electrolyte":["lipf6_OKane2022"],
   "Model option":[
         {
            "calculate discharge energy":"true",
            "particle": "Fickian diffusion",          
            "SEI":"interstitial-diffusion limited",   
            "SEI on cracks":"true",  
            "SEI film resistance":"distributed",          
            "SEI porosity change":"true",      
            "particle mechanics":("swelling and cracking", "swelling only"), 
            "loss of active material":"stress-driven", 
            "lithium plating":"partially reversible"      },
         ],
   "Inner SEI reaction proportion":[0.5,],
   "Ratio of lithium moles to SEI moles":[1,], # I have always been using 1 for solvent consumption model
   "Initial inner SEI thickness [m]":[2.5E-9,],
   "Initial outer SEI thickness [m]":[2.5E-9,],
   "SEI growth activation energy [J.mol-1]":[38000,],
   "Inner SEI lithium interstitial diffusivity [m2.s-1]":[1e-24],
   "Initial Neg SOC":[0.850,0.849],    #list(np.linspace(0.84,0.90,6)),
   "Initial Pos SOC":[0.2705, 0.270], # list(np.linspace(0.22,0.27,6)),
}
Para_dict_list = []
recursive_scan(Para_dict_list,Para_dict_All, list(Para_dict_All.keys()), {})


In [28]:
len(Para_dict_list))

27

for i in range(0,len(Para_dict_list)):
    print(Para_dict_list[i])

In [23]:
print(len(Para_dict_list))
Sol=[];Cap=[]; Neg1SOC=[]; Pos1SOC=[];
for index in range(0,len(Para_dict_list)):
    try:
        Sol_0,Cap_0, Neg1SOC_0, Pos1SOC_0 = Get_initial_cap2( Para_dict_list[index])
        Sol.append(Sol_0);Cap.append(Cap_0);
        Neg1SOC.append(Neg1SOC_0);Pos1SOC.append(Pos1SOC_0)
    except:
        print("fail in index = ", index)
    else:
        print( "succeed in index = ",  index)

4


At t = 0.122969 and h = 2.85014e-015, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.119789 and h = 8.81615e-015, the corrector convergence failed repeatedly or with |h| = hmin.


succeed in index =  0


At t = 0.122969 and h = 1.10484e-014, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.117078 and h = 7.2887e-015, the corrector convergence failed repeatedly or with |h| = hmin.


succeed in index =  1


At t = 0.119271 and h = 1.22221e-014, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.116169 and h = 2.56872e-018, the corrector convergence failed repeatedly or with |h| = hmin.


succeed in index =  2


At t = 0.119269 and h = 5.38143e-018, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.113459 and h = 4.75679e-015, the corrector convergence failed repeatedly or with |h| = hmin.


succeed in index =  3


In [24]:
print(Cap)
print(Neg1SOC)
print(Pos1SOC)

[[4.86683378485454, 4.861826795616365], [4.866943394773526, 4.857663793432799], [4.861153473329149, 4.856270967072653], [4.861262670576986, 4.8521073932633465]]
[[0.8499999999999999, 0.8473380498407365], [0.8499999999999999, 0.846606759658858], [0.8490000000000003, 0.8463620431339477], [0.8490000000000003, 0.8456304516509913]]
[[0.27050000000000013, 0.2710547989477764], [0.2700000000000001, 0.27104407372757783], [0.27050000000000013, 0.27104045682225825], [0.2700000000000001, 0.2710299201057749]]


In [25]:
# Quick plot
label = ["0","1",] 
output_variables3 = [
    "Discharge capacity [A.h]",
    "Negative electrode capacity [A.h]",
    "Positive electrode capacity [A.h]",
    "Loss of capacity to SEI [A.h]",

    
]
quick_plot = pb.QuickPlot([Sol[0],Sol[1],], output_variables3,label,variable_limits='fixed') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=40.904955501711164, step=0.40904955501711165…

In [26]:
# Quick plot
label = ["Sol_0","Sol_1"] 
output_variables3 = [
    "Negative electrode capacity [A.h]",
    "Positive electrode capacity [A.h]",
    "Loss of capacity to lithium plating [A.h]",
    "Loss of capacity to SEI [A.h]",
    "Loss of capacity to SEI on cracks [A.h]",
    
]
quick_plot = pb.QuickPlot([Sol_0,Sol_1], output_variables3,label,variable_limits='fixed') #     
quick_plot.dynamic_plot();

NameError: name 'Sol_1' is not defined

In [ ]:
print(Sol_0.cycles[1].steps[0]["Negative electrode SOC"].entries[0])
print(Sol_0.cycles[1]["Negative electrode SOC"].entries[0])
print(Sol_0.cycles[1].steps[0]["Positive electrode SOC"].entries[0])
print(Sol_0.cycles[1]["Positive electrode SOC"].entries[0])
